In [18]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import pickle as pkl
import notebook_helper
import sys
import imodelsx
import imodelsx.process_results
from mprompt.data.data import TASKS_D3
TASK_NAMES = list(TASKS_D3.keys())
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/mprompt/feb18_synthetic_sweep'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)

100%|██████████| 12/12 [00:00<00:00, 12.56it/s]


In [30]:
r['top_contains_keywords'] = r['score_contains_keywords'].apply(lambda x: x[0])
r['task_name (groundtruth)'] = r['module_num'].apply(lambda x: TASK_NAMES[int(x)].split('_')[-1])
r = r.sort_values(by='top_score_synthetic', ascending=False).round(3)

In [31]:
# display full strings
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(
        r
        .filter(['task_name (groundtruth)', 'top_explanation_init_strs', 'top_contains_keywords', 'top_score_synthetic', 'module_num'])
        # .filter(['top_score_contains_keywords', 'top_score_synthetic', 'top_explanation_init_strs', 'explanation_init_ngrams', 'module_num'])
        # .filter(['top_explanation_init_strs', 'explanation_init_ngrams', 'top_strs_added'])
        # .filter(['explanation_init_strs', 'explanation_init_ngrams',])
        .head(n=20)
    )

,task_name (groundtruth),top_explanation_init_strs,top_contains_keywords,top_score_synthetic,module_num
2,hillary,"name ""hillary""",True,0.656,16
1,politic,politics,True,0.350,53
11,god,religion,True,0.266,3
10,atheism,"religion and belief systems, specifically atheism",True,0.254,4
0,computer science,"""computer""",True,0.231,23
8,crime,crime and criminal activity,True,0.214,7
4,water,water,True,0.213,13
7,sport,sports,True,0.213,50
5,business,business,True,0.203,32
9,irony,sarcasm,True,0.173,0


In [26]:
acc = r['top_score_contains_keywords'].mean()
print('acc', acc)

acc 1.0
